# Knowledge base를 활용해서 RAG Chatbot Application 만들기

이 애플리케이션은 Amazon Bedrock Knowledge base를 활용한 최신 생성형 AI 기능을 시연하기 위해 설계되었습니다.

**Amazon Bedrock Agent**: 지능형 챗봇 기능으로, 사용자의 질문 특성에 따라 GraphRAG와 VectorRAG 검색 방식을 자동으로 선택하여 최적의 응답을 제공합니다.  
                          GraphRAG는 엔티티 간의 관계와 연결 구조를 분석하는 데 특화되어 있으며, VectorRAG는 의미적 유사성에 기반한 일반 정보 검색에 적합합니다.

**Amazon Bedrock Knowledge Base**: 일반 Vector RAG와 Graph RAG 접근 방식의 응답을 나란히 비교할 수 있는 기능을 제공합니다.  
                          이를 통해 사용자는 동일한 질문에 대해 두 검색 방식이 어떻게 다른 결과를 도출하는지 직접 확인할 수 있습니다.

이 애플리케이션은 Streamlit을 기반으로 개발되어 직관적인 웹 인터페이스를 제공하며, AWS의 Bedrock 서비스를 활용하여 Claude 3.7 Sonnet과 같은 최신 대규모 언어 모델(LLM)의 강력한 기능을 활용합니다.  
사용자는 사이드바에서 원하는 데모를 선택하고, 실시간으로 AI의 응답을 확인할 수 있습니다.

이 데모는 기업이 Amazon Bedrock을 통해 어떻게 지식 기반 AI 솔루션을 구축하고, 복잡한 질의에 대해 더 정확하고 관련성 높은 응답을 제공할 수 있는지 보여주는 실용적인 예제입니다.

## 1. 챗봇 실행 파일 생성
Amazon Bedrock Agent와 Knowledge base로 구성된 RAG 애플리케이션을 Streamlit UI 기반의 챗봇으로 테스트 해볼 수 있도록 구현.

- 준비 사항
    - Claude Sonnet 3.5 v2 모델 & Claude instant v1 모델 활성화 필요
    - 아래 코드에서 **AGENT_ID, VECTOR_RAG_KB_ID, GRAPH_RAG_KB_ID** 항목을 현재 구성한 정보로 변경 필요

In [ ]:
%%writefile ../../app.py

import streamlit as st
import boto3
import json
import time
from datetime import datetime
from botocore.client import Config

# AWS 설정 변수
AWS_REGION = "us-west-2"
AGENT_ALIAS_ID = "TSTALIASID"

AGENT_ID = "AR14QVDQII" #agent id로 수정
VECTOR_RAG_KB_ID = "VXVR4W9Y2O" #Vector KB ID로 수정
GRAPH_RAG_KB_ID = "DBXNEKHXD4" #RAG KB ID로 수정

# 페이지 설정
st.set_page_config(page_title="Amazon Bedrock Demos", layout="wide")

# 세션 상태 초기화
if "session_id" not in st.session_state:
    st.session_state.session_id = f"session_{int(time.time())}"

if "messages" not in st.session_state:
    st.session_state.messages = []

if "regular_messages" not in st.session_state:
    st.session_state.regular_messages = []

if "graph_messages" not in st.session_state:
    st.session_state.graph_messages = []

# 사이드바에 라디오 버튼 생성
with st.sidebar:
    st.header("데모 선택")
    selected_demo = st.radio(
        "사용할 데모를 선택하세요:",
        ["Amazon Bedrock Agent", "Amazon Bedrock Knowledge Base"]
    )
    
    # 선택에 따라 추가 정보 표시
    if selected_demo == "Amazon Bedrock Agent":
        st.header("Agent 정보")
        st.markdown(f"**AWS Region**: {AWS_REGION}")
        st.markdown(f"**Agent ID**: {AGENT_ID}")
        st.markdown(f"**세션 ID**: {st.session_state.session_id}")
        st.markdown(f"**현재 시간**: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    else:
        st.header("Knowledge Base 정보")
        st.markdown(f"**AWS Region**: {AWS_REGION}")
        st.markdown(f"**Knowledge Base ID**: {VECTOR_RAG_KB_ID} (Vector), {GRAPH_RAG_KB_ID} (Graph)")
        st.markdown(f"**현재 시간**: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    if st.button("새 대화 시작"):
        st.session_state.session_id = f"session_{int(time.time())}"
        st.session_state.messages = []
        st.rerun()

# Amazon Bedrock Agent 기능
def run_agent_chatbot():
    # Bedrock 클라이언트 초기화
    bedrock_agent_runtime = boto3.client(
        service_name="bedrock-agent-runtime",
        region_name=AWS_REGION
    )
    bedrock_runtime = boto3.client(
        service_name="bedrock-runtime",
        region_name=AWS_REGION
    )

    # Semantic 검색 유형 판별 함수
    def determine_search_type_semantic(query):
        prompt = f"""
        당신은 질문의 특성을 분석하여 적절한 검색 방식을 추천하는 AI입니다.
        
        다음은 두 가지 검색 방식입니다:
        1. GraphRAG: 엔티티 간의 관계, 연결 구조, 네트워크 분석, 경로 탐색, 지식 그래프 패턴 등 관계형 데이터에 적합합니다.
        2. VectorRAG: 단순 정보 검색, 키워드 기반 질의, 문서 내용 요약, 유사 개념 탐색 등 의미적 유사성에 기반한 검색에 적합합니다.
        
        다음 질문을 분석하고, "GraphRAG" 또는 "VectorRAG" 중 더 적합한 검색 방식을 결정한 후 아래 형식으로 응답해주세요:
        
        검색방식: [GraphRAG 또는 VectorRAG] \n
        근거: [이 검색 방식을 선택한 이유를 한 문장으로 설명]
        
        질문: "{query}"
        """
        
        try:
            # Amazon Bedrock의 Claude 모델을 사용
            response = bedrock_runtime.invoke_model(
                modelId="anthropic.claude-instant-v1",
                body=json.dumps({
                    "prompt": f"\n\nHuman: {prompt}\n\nAssistant:",
                    "max_tokens_to_sample": 200,
                    "temperature": 0,
                    "top_p": 0.9,
                })
            )
            
            response_body = json.loads(response["body"].read())
            response_text = response_body["completion"].strip()
            
            # 응답에서 검색 방식과 근거 추출
            lines = response_text.split('\n')
            search_type = "VectorRAG"  # 기본값
            reason = "기본 검색 방식입니다."
            
            for line in lines:
                if line.startswith("검색방식:"):
                    search_type_text = line.replace("검색방식:", "").strip()
                    if "GraphRAG" in search_type_text:
                        search_type = "GraphRAG"
                    else:
                        search_type = "VectorRAG"
                elif line.startswith("근거:"):
                    reason = line.replace("근거:", "").strip()
            
            return search_type, reason
        except Exception as e:
            st.error(f"검색 유형 결정 중 오류 발생: {str(e)}")
            return "VectorRAG", "오류 발생으로 기본 검색 방식을 사용합니다."

    # Bedrock Agent에 쿼리 전송 함수
    def query_bedrock_agent(query, search_type, reason):
        try:
            response = bedrock_agent_runtime.invoke_agent(
                agentId=AGENT_ID,
                agentAliasId=AGENT_ALIAS_ID,
                sessionId=st.session_state.session_id,
                inputText=query,
                enableTrace=False
            )
            
            # 응답 처리
            final_response = ""
            for event in response['completion']:
                if 'chunk' in event:
                    try:
                        chunk_data = json.loads(event['chunk']['bytes'].decode('utf-8'))
                        if 'content' in chunk_data:
                            final_response += chunk_data['content']
                    except json.JSONDecodeError:
                        content = event['chunk']['bytes'].decode('utf-8')
                        final_response += content
            
            # 검색 유형과 근거 표시
            final_response += "\n\n---"
            final_response += f"\n\n검색 방식: {search_type}"
            final_response += f"\n\n선택 근거: {reason}"
            final_response += "\n\n---"
            
            return final_response
        except Exception as e:
            return f"오류가 발생했습니다: {str(e)}"

    # Streamlit UI 설정
    st.title("Amazon Bedrock Agent")
    st.markdown("일반 Vector RAG와 Graph RAG를 활용한 Agentic 챗봇입니다.")

    # 채팅 기록 표시
    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])

    # 사용자 입력 처리
    if prompt := st.chat_input("질문을 입력하세요..."):
        # 사용자 메시지 표시
        st.chat_message("user").markdown(prompt)
        st.session_state.messages.append({"role": "user", "content": prompt})
        
        # 의미론적 검색 유형 결정
        with st.spinner("검색 방식 분석 중..."):
            search_type, reason = determine_search_type_semantic(prompt)
        
        # 로딩 표시
        with st.chat_message("assistant"):
            message_placeholder = st.empty()
            message_placeholder.markdown("🤔 생각 중...")
            
            # Bedrock Agent에 쿼리 전송
            response = query_bedrock_agent(prompt, search_type, reason)
            
            # 응답 표시
            message_placeholder.markdown(response)
        
        # 채팅 기록에 응답 추가
        st.session_state.messages.append({"role": "assistant", "content": response})

# Amazon Bedrock Knowledge Base 기능
def run_knowledge_base_demo():
    # Bedrock 클라이언트 초기화
    bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0})
    bedrock_runtime = boto3.client(
        service_name="bedrock-runtime",
        region_name=AWS_REGION,
        config=bedrock_config
    )
    bedrock_agent_runtime = boto3.client(
        service_name="bedrock-agent-runtime", 
        region_name=AWS_REGION,
        config=bedrock_config
    )

    # 미리 정의된 프롬프트
    predefined_prompts = [
        "Amazon은 운영 비용의 증가가 다른 재무 지표들에 어떤 영향을 미쳤나요?",
        "Amazon의 총 순매출은 시간이 지남에 따라 어떻게 변화했나요?",
        "Amazon의 온라인 소매 서비스 매출은 분기별로 어떻게 변동했나요?"
    ]

    # 페이지 헤더
    st.title("Amazon Bedrock Knowledge Base")
    st.markdown("일반 Vector RAG와 Graph RAG 접근 방식 간의 응답 비교")

    # Knowledge Base에서 컨텍스트 검색 함수
    def retrieve_from_knowledge_base(query, kb_id, number_of_results=3):
        try:
            # KB ID에 따라 검색 유형 결정
            search_type = "SEMANTIC" if kb_id == GRAPH_RAG_KB_ID else "HYBRID"
            
            # Knowledge Base에서 컨텍스트 검색
            response = bedrock_agent_runtime.retrieve(
                retrievalQuery={
                    'text': query
                },
                knowledgeBaseId=kb_id,
                retrievalConfiguration={
                    'vectorSearchConfiguration': {
                        'numberOfResults': number_of_results,
                        'overrideSearchType': search_type
                    }
                }
            )
            
            # 결과에서 컨텍스트 추출
            contexts = []
            if 'retrievalResults' in response:
                for result in response['retrievalResults']:
                    if 'content' in result and 'text' in result['content']:
                        contexts.append(result['content']['text'])
            
            return contexts
        
        except Exception as e:
            return f"Error retrieving from Knowledge Base: {str(e)}"

    # Knowledge Base 쿼리 함수
    def query_knowledge_base(query, kb_id):
        try:
            # Knowledge Base에서 관련 컨텍스트 검색
            contexts = retrieve_from_knowledge_base(query, kb_id)
            
            if isinstance(contexts, str) and contexts.startswith("Error"):
                return contexts  # 오류 메시지 반환
            
            # 검색된 컨텍스트로 프롬프트 포맷팅
            prompt = f"""
Human: You are an advisor AI system, and provides answers to questions by using fact based when possible.
You're a helpful assistant who loves to respond in Korean.
Use the following pieces of information to provide a detailed answer to the question enclosed in <question> tags.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

<context>
{contexts}
</context>

<question>
{query}
</question>

The response should be specific and use statistics or numbers when possible.

A:"""

            # Claude 3.7 Sonnet 사용
            model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0" 
            
            response = bedrock_runtime.invoke_model(
                body=json.dumps({
                    "anthropic_version": "bedrock-2023-05-31",
                    "max_tokens": 8192,
                    "messages": [{"role": "user", "content": [{"type": "text", "text": prompt}]}],
                    "temperature": 0.0,
                    "top_p": 0
                }),
                modelId=model_id,
                accept="application/json",
                contentType="application/json"
            )
            
            # 응답 추출
            response_body = json.loads(response.get('body').read())
            response_text = response_body.get('content')[0]['text']
            
            return response_text
        
        except Exception as e:
            return f"Error querying Knowledge Base: {str(e)}"

    # 응답을 위한 두 개의 열 생성
    col1, col2 = st.columns(2)

    # 채팅 기록 표시
    with col1:
        st.subheader("Vector RAG")
        for message in st.session_state.regular_messages:
            if message["role"] == "user":
                st.chat_message("user").write(message["content"])
            else:
                st.chat_message("assistant").write(message["content"])

    with col2:
        st.subheader("Graph RAG")
        for message in st.session_state.graph_messages:
            if message["role"] == "user":
                st.chat_message("user").write(message["content"])
            else:
                st.chat_message("assistant").write(message["content"])

    # 미리 정의된 프롬프트에서 사용자 선택
    selected_prompt_index = st.selectbox("질문을 선택하세요:", options=range(len(predefined_prompts)), format_func=lambda i: predefined_prompts[i])
    
    # 선택한 프롬프트 제출 버튼
    if st.button("질문 제출"):
        # 선택한 프롬프트 가져오기 - 이 부분이 수정되었습니다
        user_query = predefined_prompts[selected_prompt_index]
        
        # 이전 메시지 지우기
        st.session_state.regular_messages = []
        st.session_state.graph_messages = []
        
        # 두 채팅 기록에 사용자 메시지 추가
        st.session_state.regular_messages.append({"role": "user", "content": user_query})
        st.session_state.graph_messages.append({"role": "user", "content": user_query})
        
        # 두 열에 사용자 메시지 표시
        with col1:
            st.chat_message("user").write(user_query)
        with col2:
            st.chat_message("user").write(user_query)
        
        # 두 지식 베이스 쿼리
        regular_response = query_knowledge_base(user_query, VECTOR_RAG_KB_ID)
        graph_response = query_knowledge_base(user_query, GRAPH_RAG_KB_ID)
        
        # 채팅 기록에 어시스턴트 응답 추가
        st.session_state.regular_messages.append({"role": "assistant", "content": regular_response})
        st.session_state.graph_messages.append({"role": "assistant", "content": graph_response})
        
        # 어시스턴트 응답 표시
        with col1:
            with st.chat_message("assistant"):
                st.write(regular_response)
        
        with col2:
            with st.chat_message("assistant"):
                st.write(graph_response)
        
        # UI 업데이트를 위한 재실행
        st.rerun()

# 라디오 버튼 선택에 따라 선택된 데모 실행
if selected_demo == "Amazon Bedrock Agent":
    run_agent_chatbot()
else:
    run_knowledge_base_demo()

## 2. 챗봇 실행을 위한 requirements 설치 및 실행

- 아래 코드를 실행 하고 현재 SageMaker 노트북이 열려있는 웹 브라우저의 URL을 복사하고 URL 마지막 부분을 아래와 같이 수정 후 실행 가능

```python
https://rag-test.studio.us-west-2.sagemaker.aws/jupyterlab/default/[추가]proxy/8080/
```

In [ ]:
# 패키지 설치
!pip install -r ../requirements.txt

In [ ]:
# 앱 실행
!streamlit run ../../app.py --server.port 8080

---
# Option - 코드 이해하기

## 1. 라이브러리 임포트 및 설정
필요한 라이브러리를 임포트하고 AWS 리소스 ID와 리전 설정을 위한 변수들을 정의합니다.

In [ ]:
import streamlit as st
import boto3
import json
import time
from datetime import datetime
from botocore.client import Config

# AWS 설정 변수
AWS_REGION = "us-west-2" #사용중인 Region으로 변경
AGENT_ID = "AR14QVDQII" #Agent_ID로 수정
AGENT_ALIAS_ID = "TSTALIASID" #Agent alias id는 현재 값으로 유지 "TSTALIASID"
VECTOR_RAG_KB_ID = "VXVR4W9Y2O" #VectorRAG KB ID로 수정
GRAPH_RAG_KB_ID = "DBXNEKHXD4" #GraphRAG KB ID로 수정

## 2. 페이지 설정 및 세션 상태 초기화
Streamlit 페이지를 설정하고 대화 세션을 관리하기 위한 세션 상태 변수들을 초기화합니다.



In [ ]:
# 페이지 설정
st.set_page_config(page_title="Amazon Bedrock RAG Demos", layout="wide")

# 세션 상태 초기화
if "session_id" not in st.session_state:
    st.session_state.session_id = f"session_{int(time.time())}"

if "messages" not in st.session_state:
    st.session_state.messages = []

if "regular_messages" not in st.session_state:
    st.session_state.regular_messages = []

if "graph_messages" not in st.session_state:
    st.session_state.graph_messages = []


## 3. 사이드바 UI 구성
사이드바에 데모 선택 라디오 버튼과 관련 정보를 표시합니다. 선택된 데모에 따라 다른 정보를 보여줍니다.

In [ ]:
# 사이드바에 라디오 버튼 생성
with st.sidebar:
    st.header("데모 선택")
    selected_demo = st.radio(
        "사용할 데모를 선택하세요:",
        ["Amazon Bedrock Agent", "Amazon Bedrock Knowledge Base"]
    )
    
    # 선택에 따라 추가 정보 표시
    if selected_demo == "Amazon Bedrock Agent":
        st.header("Agent 정보")
        st.markdown(f"**AWS Region**: {AWS_REGION}")
        st.markdown(f"**Agent ID**: {AGENT_ID}")
        st.markdown(f"**세션 ID**: {st.session_state.session_id}")
        st.markdown(f"**현재 시간**: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    else:
        st.header("Knowledge Base 정보")
        st.markdown(f"**AWS Region**: {AWS_REGION}")
        st.markdown(f"**Knowledge Base ID**: {VECTOR_RAG_KB_ID} (Vector), {GRAPH_RAG_KB_ID} (Graph)")
        st.markdown(f"**현재 시간**: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    if st.button("새 대화 시작"):
        st.session_state.session_id = f"session_{int(time.time())}"
        st.session_state.messages = []
        st.rerun()


## 4. Amazon Bedrock Agent RAG 챗봇
Amazon Bedrock Agent를 활용한 챗봇 기능으로, 질문 특성에 따라 GraphRAG와 VectorRAG 검색 방식을 자동으로 선택합니다.

In [ ]:
def run_agent_chatbot():
    # Bedrock 클라이언트 초기화
    bedrock_agent_runtime = boto3.client(
        service_name="bedrock-agent-runtime",
        region_name=AWS_REGION
    )
    bedrock_runtime = boto3.client(
        service_name="bedrock-runtime",
        region_name=AWS_REGION
    )

    # Semantic 검색 유형 판별 함수
    def determine_search_type_semantic(query):
        prompt = f"""
        당신은 질문의 특성을 분석하여 적절한 검색 방식을 추천하는 AI입니다.
        
        다음은 두 가지 검색 방식입니다:
        1. GraphRAG: 엔티티 간의 관계, 연결 구조, 네트워크 분석, 경로 탐색, 지식 그래프 패턴 등 관계형 데이터에 적합합니다.
        2. VectorRAG: 단순 정보 검색, 키워드 기반 질의, 문서 내용 요약, 유사 개념 탐색 등 의미적 유사성에 기반한 검색에 적합합니다.
        
        다음 질문을 분석하고, "GraphRAG" 또는 "VectorRAG" 중 더 적합한 검색 방식을 결정한 후 아래 형식으로 응답해주세요:
        
        검색방식: [GraphRAG 또는 VectorRAG] \n
        근거: [이 검색 방식을 선택한 이유를 한 문장으로 설명]
        
        질문: "{query}"
        """
        
        try:
            # Amazon Bedrock의 Claude 모델을 사용
            response = bedrock_runtime.invoke_model(
                modelId="anthropic.claude-instant-v1",
                body=json.dumps({
                    "prompt": f"\n\nHuman: {prompt}\n\nAssistant:",
                    "max_tokens_to_sample": 200,
                    "temperature": 0,
                    "top_p": 0.9,
                })
            )
            
            response_body = json.loads(response["body"].read())
            response_text = response_body["completion"].strip()
            
            # 응답에서 검색 방식과 근거 추출
            lines = response_text.split('\n')
            search_type = "VectorRAG"  # 기본값
            reason = "기본 검색 방식입니다."
            
            for line in lines:
                if line.startswith("검색방식:"):
                    search_type_text = line.replace("검색방식:", "").strip()
                    if "GraphRAG" in search_type_text:
                        search_type = "GraphRAG"
                    else:
                        search_type = "VectorRAG"
                elif line.startswith("근거:"):
                    reason = line.replace("근거:", "").strip()
            
            return search_type, reason
        except Exception as e:
            st.error(f"검색 유형 결정 중 오류 발생: {str(e)}")
            return "VectorRAG", "오류 발생으로 기본 검색 방식을 사용합니다."

    # Bedrock Agent에 쿼리 전송 함수
    def query_bedrock_agent(query, search_type, reason):
        try:
            response = bedrock_agent_runtime.invoke_agent(
                agentId=AGENT_ID,
                agentAliasId=AGENT_ALIAS_ID,
                sessionId=st.session_state.session_id,
                inputText=query,
                enableTrace=False
            )
            
            # 응답 처리
            final_response = ""
            for event in response['completion']:
                if 'chunk' in event:
                    try:
                        chunk_data = json.loads(event['chunk']['bytes'].decode('utf-8'))
                        if 'content' in chunk_data:
                            final_response += chunk_data['content']
                    except json.JSONDecodeError:
                        content = event['chunk']['bytes'].decode('utf-8')
                        final_response += content
            
            # 검색 유형과 근거 표시
            final_response += "\n\n---"
            final_response += f"\n\n검색 방식: {search_type}"
            final_response += f"\n\n선택 근거: {reason}"
            final_response += "\n\n---"
            
            return final_response
        except Exception as e:
            return f"오류가 발생했습니다: {str(e)}"

    # Streamlit UI 설정
    st.title("Amazon Bedrock Agent")
    st.markdown("GraphRAG와 VectorRAG를 활용한 지능형 챗봇입니다.")

    # 채팅 기록 표시
    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])

    # 사용자 입력 처리
    if prompt := st.chat_input("질문을 입력하세요..."):
        # 사용자 메시지 표시
        st.chat_message("user").markdown(prompt)
        st.session_state.messages.append({"role": "user", "content": prompt})
        
        # 의미론적 검색 유형 결정
        with st.spinner("검색 방식 분석 중..."):
            search_type, reason = determine_search_type_semantic(prompt)
        
        # 로딩 표시
        with st.chat_message("assistant"):
            message_placeholder = st.empty()
            message_placeholder.markdown("🤔 생각 중...")
            
            # Bedrock Agent에 쿼리 전송
            response = query_bedrock_agent(prompt, search_type, reason)
            
            # 응답 표시
            message_placeholder.markdown(response)
        
        # 채팅 기록에 응답 추가
        st.session_state.messages.append({"role": "assistant", "content": response})


## 5. Amazon Bedrock Knowledge Base 비교 챗봇
Vector RAG와 Graph RAG 접근 방식의 응답을 나란히 비교할 수 있는 기능을 제공합니다.



In [ ]:
def run_knowledge_base_demo():
    # Bedrock 클라이언트 초기화
    bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0})
    bedrock_runtime = boto3.client(
        service_name="bedrock-runtime",
        region_name=AWS_REGION,
        config=bedrock_config
    )
    bedrock_agent_runtime = boto3.client(
        service_name="bedrock-agent-runtime", 
        region_name=AWS_REGION,
        config=bedrock_config
    )

    # 미리 정의된 프롬프트
    predefined_prompts = [
        "Amazon은 운영 비용의 증가가 다른 재무 지표들에 어떤 영향을 미쳤나요?",
        "Amazon의 총 순매출은 시간이 지남에 따라 어떻게 변화했나요?",
        "Amazon의 온라인 소매 서비스 매출은 분기별로 어떻게 변동했나요?"
    ]

    # 페이지 헤더
    st.title("Amazon Bedrock Knowledge Base")
    st.markdown("일반 Vector RAG와 Graph RAG 접근 방식 간의 응답 비교")

    # Knowledge Base에서 컨텍스트 검색 함수
    def retrieve_from_knowledge_base(query, kb_id, number_of_results=3):
        try:
            # KB ID에 따라 검색 유형 결정
            search_type = "SEMANTIC" if kb_id == GRAPH_RAG_KB_ID else "HYBRID"
            
            # Knowledge Base에서 컨텍스트 검색
            response = bedrock_agent_runtime.retrieve(
                retrievalQuery={
                    'text': query
                },
                knowledgeBaseId=kb_id,
                retrievalConfiguration={
                    'vectorSearchConfiguration': {
                        'numberOfResults': number_of_results,
                        'overrideSearchType': search_type
                    }
                }
            )
            
            # 결과에서 컨텍스트 추출
            contexts = []
            if 'retrievalResults' in response:
                for result in response['retrievalResults']:
                    if 'content' in result and 'text' in result['content']:
                        contexts.append(result['content']['text'])
            
            return contexts
        
        except Exception as e:
            return f"Error retrieving from Knowledge Base: {str(e)}"

    # Knowledge Base 쿼리 함수
    def query_knowledge_base(query, kb_id):
        try:
            # Knowledge Base에서 관련 컨텍스트 검색
            contexts = retrieve_from_knowledge_base(query, kb_id)
            
            if isinstance(contexts, str) and contexts.startswith("Error"):
                return contexts  # 오류 메시지 반환
            
            # 검색된 컨텍스트로 프롬프트 포맷팅
            prompt = f"""
Human: You are an advisor AI system, and provides answers to questions by using fact based when possible.
You're a helpful assistant who loves to respond in Korean.
Use the following pieces of information to provide a detailed answer to the question enclosed in <question> tags.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

<context>
{contexts}
</context>

<question>
{query}
</question>

The response should be specific and use statistics or numbers when possible.

A:"""

            # Claude 5 Sonnet 사용
            model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0" 
            
            response = bedrock_runtime.invoke_model(
                body=json.dumps({
                    "anthropic_version": "bedrock-2023-05-31",
                    "max_tokens": 8192,
                    "messages": [{"role": "user", "content": [{"type": "text", "text": prompt}]}],
                    "temperature": 0.0,
                    "top_p": 0
                }),
                modelId=model_id,
                accept="application/json",
                contentType="application/json"
            )
            
            # 응답 추출
            response_body = json.loads(response.get('body').read())
            response_text = response_body.get('content')[0]['text']
            
            return response_text
        
        except Exception as e:
            return f"Error querying Knowledge Base: {str(e)}"

    # 응답을 위한 두 개의 열 생성
    col1, col2 = st.columns(2)

    # 채팅 기록 표시
    with col1:
        st.subheader("Vector RAG")
        for message in st.session_state.regular_messages:
            if message["role"] == "user":
                st.chat_message("user").write(message["content"])
            else:
                st.chat_message("assistant").write(message["content"])

    with col2:
        st.subheader("Graph RAG")
        for message in st.session_state.graph_messages:
            if message["role"] == "user":
                st.chat_message("user").write(message["content"])
            else:
                st.chat_message("assistant").write(message["content"])

    # 미리 정의된 프롬프트에서 사용자 선택
    selected_prompt_index = st.selectbox("질문을 선택하세요:", options=range(len(predefined_prompts)), format_func=lambda i: predefined_prompts[i])
    
    # 선택한 프롬프트 제출 버튼
    if st.button("질문 제출"):
        user_query = predefined_prompts[selected_prompt_index]
        
        # 이전 메시지 지우기
        st.session_state.regular_messages = []
        st.session_state.graph_messages = []
        
        # 두 채팅 기록에 사용자 메시지 추가
        st.session_state.regular_messages.append({"role": "user", "content": user_query})
        st.session_state.graph_messages.append({"role": "user", "content": user_query})
        
        # 두 열에 사용자 메시지 표시
        with col1:
            st.chat_message("user").write(user_query)
        with col2:
            st.chat_message("user").write(user_query)
        
        # 두 지식 베이스 쿼리
        regular_response = query_knowledge_base(user_query, VECTOR_RAG_KB_ID)
        graph_response = query_knowledge_base(user_query, GRAPH_RAG_KB_ID)
        
        # 채팅 기록에 어시스턴트 응답 추가
        st.session_state.regular_messages.append({"role": "assistant", "content": regular_response})
        st.session_state.graph_messages.append({"role": "assistant", "content": graph_response})
        
        # 어시스턴트 응답 표시
        with col1:
            with st.chat_message("assistant"):
                st.write(regular_response)
        
        with col2:
            with st.chat_message("assistant"):
                st.write(graph_response)
        
        # UI 업데이트를 위한 재실행
        st.rerun()


## 6. 챗봇 실행 메인 로직
사용자가 선택한 데모에 따라 해당하는 기능을 실행하는 메인 로직입니다.

In [ ]:
# 라디오 버튼 선택에 따라 선택된 데모 실행
if selected_demo == "Amazon Bedrock Agent":
    run_agent_chatbot()
else:
    run_knowledge_base_demo()

## 7. 챗봇 실행 파일 생성
Amazon Bedrock Agent와 Knowledge base로 구성된 RAG 애플리케이션을 Streamlit UI 기반의 챗봇으로 테스트 해볼 수 있도록 구현.

- 준비 사항
    - Claude Sonnet 3.5 v2 모델 & Claude instant v1 모델 활성화 필요
    - 아래 코드에서 **AGENT_ID, VECTOR_RAG_KB_ID, GRAPH_RAG_KB_ID** 항목을 현재 구성한 정보로 변경 필요

In [ ]:
%%writefile ../app.py

import streamlit as st
import boto3
import json
import time
from datetime import datetime
from botocore.client import Config

# AWS 설정 변수
AWS_REGION = "us-west-2"
AGENT_ALIAS_ID = "TSTALIASID"

AGENT_ID = "AR14QVDQII" #agent id로 수정
VECTOR_RAG_KB_ID = "VXVR4W9Y2O" #Vector KB ID로 수정
GRAPH_RAG_KB_ID = "DBXNEKHXD4" #RAG KB ID로 수정

# 페이지 설정
st.set_page_config(page_title="Amazon Bedrock Demos", layout="wide")

# 세션 상태 초기화
if "session_id" not in st.session_state:
    st.session_state.session_id = f"session_{int(time.time())}"

if "messages" not in st.session_state:
    st.session_state.messages = []

if "regular_messages" not in st.session_state:
    st.session_state.regular_messages = []

if "graph_messages" not in st.session_state:
    st.session_state.graph_messages = []

# 사이드바에 라디오 버튼 생성
with st.sidebar:
    st.header("데모 선택")
    selected_demo = st.radio(
        "사용할 데모를 선택하세요:",
        ["Amazon Bedrock Agent", "Amazon Bedrock Knowledge Base"]
    )
    
    # 선택에 따라 추가 정보 표시
    if selected_demo == "Amazon Bedrock Agent":
        st.header("Agent 정보")
        st.markdown(f"**AWS Region**: {AWS_REGION}")
        st.markdown(f"**Agent ID**: {AGENT_ID}")
        st.markdown(f"**세션 ID**: {st.session_state.session_id}")
        st.markdown(f"**현재 시간**: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    else:
        st.header("Knowledge Base 정보")
        st.markdown(f"**AWS Region**: {AWS_REGION}")
        st.markdown(f"**Knowledge Base ID**: {VECTOR_RAG_KB_ID} (Vector), {GRAPH_RAG_KB_ID} (Graph)")
        st.markdown(f"**현재 시간**: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    if st.button("새 대화 시작"):
        st.session_state.session_id = f"session_{int(time.time())}"
        st.session_state.messages = []
        st.rerun()

# Amazon Bedrock Agent 기능
def run_agent_chatbot():
    # Bedrock 클라이언트 초기화
    bedrock_agent_runtime = boto3.client(
        service_name="bedrock-agent-runtime",
        region_name=AWS_REGION
    )
    bedrock_runtime = boto3.client(
        service_name="bedrock-runtime",
        region_name=AWS_REGION
    )

    # Semantic 검색 유형 판별 함수
    def determine_search_type_semantic(query):
        prompt = f"""
        당신은 질문의 특성을 분석하여 적절한 검색 방식을 추천하는 AI입니다.
        
        다음은 두 가지 검색 방식입니다:
        1. GraphRAG: 엔티티 간의 관계, 연결 구조, 네트워크 분석, 경로 탐색, 지식 그래프 패턴 등 관계형 데이터에 적합합니다.
        2. VectorRAG: 단순 정보 검색, 키워드 기반 질의, 문서 내용 요약, 유사 개념 탐색 등 의미적 유사성에 기반한 검색에 적합합니다.
        
        다음 질문을 분석하고, "GraphRAG" 또는 "VectorRAG" 중 더 적합한 검색 방식을 결정한 후 아래 형식으로 응답해주세요:
        
        검색방식: [GraphRAG 또는 VectorRAG] \n
        근거: [이 검색 방식을 선택한 이유를 한 문장으로 설명]
        
        질문: "{query}"
        """
        
        try:
            # Amazon Bedrock의 Claude 모델을 사용
            response = bedrock_runtime.invoke_model(
                modelId="anthropic.claude-instant-v1",
                body=json.dumps({
                    "prompt": f"\n\nHuman: {prompt}\n\nAssistant:",
                    "max_tokens_to_sample": 200,
                    "temperature": 0,
                    "top_p": 0.9,
                })
            )
            
            response_body = json.loads(response["body"].read())
            response_text = response_body["completion"].strip()
            
            # 응답에서 검색 방식과 근거 추출
            lines = response_text.split('\n')
            search_type = "VectorRAG"  # 기본값
            reason = "기본 검색 방식입니다."
            
            for line in lines:
                if line.startswith("검색방식:"):
                    search_type_text = line.replace("검색방식:", "").strip()
                    if "GraphRAG" in search_type_text:
                        search_type = "GraphRAG"
                    else:
                        search_type = "VectorRAG"
                elif line.startswith("근거:"):
                    reason = line.replace("근거:", "").strip()
            
            return search_type, reason
        except Exception as e:
            st.error(f"검색 유형 결정 중 오류 발생: {str(e)}")
            return "VectorRAG", "오류 발생으로 기본 검색 방식을 사용합니다."

    # Bedrock Agent에 쿼리 전송 함수
    def query_bedrock_agent(query, search_type, reason):
        try:
            response = bedrock_agent_runtime.invoke_agent(
                agentId=AGENT_ID,
                agentAliasId=AGENT_ALIAS_ID,
                sessionId=st.session_state.session_id,
                inputText=query,
                enableTrace=False
            )
            
            # 응답 처리
            final_response = ""
            for event in response['completion']:
                if 'chunk' in event:
                    try:
                        chunk_data = json.loads(event['chunk']['bytes'].decode('utf-8'))
                        if 'content' in chunk_data:
                            final_response += chunk_data['content']
                    except json.JSONDecodeError:
                        content = event['chunk']['bytes'].decode('utf-8')
                        final_response += content
            
            # 검색 유형과 근거 표시
            final_response += "\n\n---"
            final_response += f"\n\n검색 방식: {search_type}"
            final_response += f"\n\n선택 근거: {reason}"
            final_response += "\n\n---"
            
            return final_response
        except Exception as e:
            return f"오류가 발생했습니다: {str(e)}"

    # Streamlit UI 설정
    st.title("Amazon Bedrock Agent")
    st.markdown("일반 Vector RAG와 Graph RAG를 활용한 Agentic 챗봇입니다.")

    # 채팅 기록 표시
    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])

    # 사용자 입력 처리
    if prompt := st.chat_input("질문을 입력하세요..."):
        # 사용자 메시지 표시
        st.chat_message("user").markdown(prompt)
        st.session_state.messages.append({"role": "user", "content": prompt})
        
        # 의미론적 검색 유형 결정
        with st.spinner("검색 방식 분석 중..."):
            search_type, reason = determine_search_type_semantic(prompt)
        
        # 로딩 표시
        with st.chat_message("assistant"):
            message_placeholder = st.empty()
            message_placeholder.markdown("🤔 생각 중...")
            
            # Bedrock Agent에 쿼리 전송
            response = query_bedrock_agent(prompt, search_type, reason)
            
            # 응답 표시
            message_placeholder.markdown(response)
        
        # 채팅 기록에 응답 추가
        st.session_state.messages.append({"role": "assistant", "content": response})

# Amazon Bedrock Knowledge Base 기능
def run_knowledge_base_demo():
    # Bedrock 클라이언트 초기화
    bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0})
    bedrock_runtime = boto3.client(
        service_name="bedrock-runtime",
        region_name=AWS_REGION,
        config=bedrock_config
    )
    bedrock_agent_runtime = boto3.client(
        service_name="bedrock-agent-runtime", 
        region_name=AWS_REGION,
        config=bedrock_config
    )

    # 미리 정의된 프롬프트
    predefined_prompts = [
        "Amazon은 운영 비용의 증가가 다른 재무 지표들에 어떤 영향을 미쳤나요?",
        "Amazon의 총 순매출은 시간이 지남에 따라 어떻게 변화했나요?",
        "Amazon의 온라인 소매 서비스 매출은 분기별로 어떻게 변동했나요?"
    ]

    # 페이지 헤더
    st.title("Amazon Bedrock Knowledge Base")
    st.markdown("일반 Vector RAG와 Graph RAG 접근 방식 간의 응답 비교")

    # Knowledge Base에서 컨텍스트 검색 함수
    def retrieve_from_knowledge_base(query, kb_id, number_of_results=3):
        try:
            # KB ID에 따라 검색 유형 결정
            search_type = "SEMANTIC" if kb_id == GRAPH_RAG_KB_ID else "HYBRID"
            
            # Knowledge Base에서 컨텍스트 검색
            response = bedrock_agent_runtime.retrieve(
                retrievalQuery={
                    'text': query
                },
                knowledgeBaseId=kb_id,
                retrievalConfiguration={
                    'vectorSearchConfiguration': {
                        'numberOfResults': number_of_results,
                        'overrideSearchType': search_type
                    }
                }
            )
            
            # 결과에서 컨텍스트 추출
            contexts = []
            if 'retrievalResults' in response:
                for result in response['retrievalResults']:
                    if 'content' in result and 'text' in result['content']:
                        contexts.append(result['content']['text'])
            
            return contexts
        
        except Exception as e:
            return f"Error retrieving from Knowledge Base: {str(e)}"

    # Knowledge Base 쿼리 함수
    def query_knowledge_base(query, kb_id):
        try:
            # Knowledge Base에서 관련 컨텍스트 검색
            contexts = retrieve_from_knowledge_base(query, kb_id)
            
            if isinstance(contexts, str) and contexts.startswith("Error"):
                return contexts  # 오류 메시지 반환
            
            # 검색된 컨텍스트로 프롬프트 포맷팅
            prompt = f"""
Human: You are an advisor AI system, and provides answers to questions by using fact based when possible.
You're a helpful assistant who loves to respond in Korean.
Use the following pieces of information to provide a detailed answer to the question enclosed in <question> tags.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

<context>
{contexts}
</context>

<question>
{query}
</question>

The response should be specific and use statistics or numbers when possible.

A:"""

            # Claude 3.5 Sonnet 사용
            model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0" 
            
            response = bedrock_runtime.invoke_model(
                body=json.dumps({
                    "anthropic_version": "bedrock-2023-05-31",
                    "max_tokens": 8192,
                    "messages": [{"role": "user", "content": [{"type": "text", "text": prompt}]}],
                    "temperature": 0.0,
                    "top_p": 0
                }),
                modelId=model_id,
                accept="application/json",
                contentType="application/json"
            )
            
            # 응답 추출
            response_body = json.loads(response.get('body').read())
            response_text = response_body.get('content')[0]['text']
            
            return response_text
        
        except Exception as e:
            return f"Error querying Knowledge Base: {str(e)}"

    # 응답을 위한 두 개의 열 생성
    col1, col2 = st.columns(2)

    # 채팅 기록 표시
    with col1:
        st.subheader("Vector RAG")
        for message in st.session_state.regular_messages:
            if message["role"] == "user":
                st.chat_message("user").write(message["content"])
            else:
                st.chat_message("assistant").write(message["content"])

    with col2:
        st.subheader("Graph RAG")
        for message in st.session_state.graph_messages:
            if message["role"] == "user":
                st.chat_message("user").write(message["content"])
            else:
                st.chat_message("assistant").write(message["content"])

    # 미리 정의된 프롬프트에서 사용자 선택
    selected_prompt_index = st.selectbox("질문을 선택하세요:", options=range(len(predefined_prompts)), format_func=lambda i: predefined_prompts[i])
    
    # 선택한 프롬프트 제출 버튼
    if st.button("질문 제출"):
        # 선택한 프롬프트 가져오기 - 이 부분이 수정되었습니다
        user_query = predefined_prompts[selected_prompt_index]
        
        # 이전 메시지 지우기
        st.session_state.regular_messages = []
        st.session_state.graph_messages = []
        
        # 두 채팅 기록에 사용자 메시지 추가
        st.session_state.regular_messages.append({"role": "user", "content": user_query})
        st.session_state.graph_messages.append({"role": "user", "content": user_query})
        
        # 두 열에 사용자 메시지 표시
        with col1:
            st.chat_message("user").write(user_query)
        with col2:
            st.chat_message("user").write(user_query)
        
        # 두 지식 베이스 쿼리
        regular_response = query_knowledge_base(user_query, VECTOR_RAG_KB_ID)
        graph_response = query_knowledge_base(user_query, GRAPH_RAG_KB_ID)
        
        # 채팅 기록에 어시스턴트 응답 추가
        st.session_state.regular_messages.append({"role": "assistant", "content": regular_response})
        st.session_state.graph_messages.append({"role": "assistant", "content": graph_response})
        
        # 어시스턴트 응답 표시
        with col1:
            with st.chat_message("assistant"):
                st.write(regular_response)
        
        with col2:
            with st.chat_message("assistant"):
                st.write(graph_response)
        
        # UI 업데이트를 위한 재실행
        st.rerun()

# 라디오 버튼 선택에 따라 선택된 데모 실행
if selected_demo == "Amazon Bedrock Agent":
    run_agent_chatbot()
else:
    run_knowledge_base_demo()


Overwriting ../app.py


## 8. 챗봇 실행을 위한 requirements 설치 및 실행

- 아래 코드를 실행 하고 현재 SageMaker 노트북이 열려있는 웹 브라우저의 URL을 복사하고 URL 마지막 부분을 아래와 같이 수정 후 실행 가능
    - 예시 - https://rag-test.studio.us-west-2.sagemaker.aws/jupyterlab/**default/proxy/8080/**

In [ ]:
# 패키지 설치
pip install -r requirements.txt

# 앱 실행
streamlit run ../app.py --server.port 8080